In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# Store Part I results into DataFrame
# Load the csv exported in Part I to a DataFrame
# Create DataFrame from WeatherPy csv 
weather_info = pd.read_csv("output_file.csv")
weather_info

,City,Cloudiness,Country,Humidity,Latitude,Longitude,Max Temperature,Wind Speeds
0,El Puerto de Santa María,0,ES,93,36.59,-6.23,57.20,3.36
1,Port,94,CD,60,-7.03,29.76,73.56,5.66
2,Rikitea,0,PF,67,-23.12,-134.97,76.08,7.36
3,Hobart,75,AU,67,-42.88,147.33,64.99,8.05
4,Hermanus,0,ZA,80,-34.42,19.23,57.00,5.01
...,...,...,...,...,...,...,...,...
542,Zambezi,13,ZM,65,-13.54,23.10,66.99,2.10
543,Kieta,76,PG,64,-6.22,155.63,86.27,4.50
544,Daru,44,PG,69,-9.08,143.21,84.85,13.31
545,Elliot,100,ZA,94,-31.33,27.85,49.35,3.24


In [3]:
# Humidity Heatmap
# Configure gmaps.
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight.
# Set up map
location = weather_info[["Latitude", "Longitude"]].astype(float)
hum = weather_info["Humidity"].astype(float)

fig = gmaps.figure(map_type="SATELLITE")
heat = gmaps.heatmap_layer(location, weights=hum, dissipating=False, max_intensity=100,point_radius=2)

# Add Heatmap layer to map.
fig.add_layer(heat)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# Create new DataFrame fitting weather criteria
# Narrow down the cities to fit weather conditions.
# Drop any rows will null values.
perfect = weather_info.loc[(weather_info["Max Temperature"]>= 70)&(weather_info["Max Temperature"]<=80)&(weather_info["Wind Speeds"]<10)&(weather_info["Cloudiness"]==0)]
hotel_df = perfect.dropna()
hotel_df

,City,Cloudiness,Country,Humidity,Latitude,Longitude,Max Temperature,Wind Speeds
2,Rikitea,0,PF,67,-23.12,-134.97,76.08,7.36
8,Nelson,0,NZ,39,-41.28,173.28,73.99,4.00
36,Airai,0,TL,37,-8.93,125.41,75.69,3.24
154,Hwange,0,ZW,32,-18.37,26.50,71.91,5.39
286,Maceió,0,BR,94,-9.67,-35.74,75.20,4.70
326,Salalah,0,OM,37,17.02,54.09,71.60,3.36
341,Challakere,0,IN,76,14.32,76.65,70.36,2.21
342,Verāval,0,IN,46,20.90,70.37,77.95,9.51
402,Neuquén,0,AR,43,-38.95,-68.06,71.60,2.24
419,Bonthe,0,SL,84,7.53,-12.51,78.30,4.27


In [5]:
# Hotel Map
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""


# Set parameters to search for hotels with 5000 meters.

param = {"radius": 5000,
         "types": "lodging",
         "key": g_key
        }

for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    param["location"]=f"{lat},{lng}"
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_name = requests.get(url, params=param).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][3]["name"]
        
    except (KeyError, IndexError):
        print("Skipping")
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.
hotel_df

Skipping
Skipping
Skipping
Skipping
Skipping


,City,Cloudiness,Country,Humidity,Latitude,Longitude,Max Temperature,Wind Speeds,Hotel Name
2,Rikitea,0,PF,67,-23.12,-134.97,76.08,7.36,
8,Nelson,0,NZ,39,-41.28,173.28,73.99,4.00,Tasman Bay Backpackers
36,Airai,0,TL,37,-8.93,125.41,75.69,3.24,
154,Hwange,0,ZW,32,-18.37,26.50,71.91,5.39,Decadent guest house
286,Maceió,0,BR,94,-9.67,-35.74,75.20,4.70,Hotel Ponta Verde Maceió
326,Salalah,0,OM,37,17.02,54.09,71.60,3.36,Muscat International Hotel plaza Salalah
341,Challakere,0,IN,76,14.32,76.65,70.36,2.21,Jairam Lodge
342,Verāval,0,IN,46,20.90,70.37,77.95,9.51,Hotel Woodland Somnath
402,Neuquén,0,AR,43,-38.95,-68.06,71.60,2.24,Hostel Portal de Sueños
419,Bonthe,0,SL,84,7.53,-12.51,78.30,4.27,


In [6]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [7]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info )
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Save figure
plt.savefig('../WeatherPy/Images/map.png')

<Figure size 432x288 with 0 Axes>